* Ссылки 
  * [Разведочный анализ данных](https://www.kaggle.com/emstrakhov/eda-with-pandas).<br>

In [1]:
from sklearn.model_selection import train_test_split
import warnings
from functools import reduce
import seaborn as sns
import os
import csv
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
warnings.filterwarnings("ignore")


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

sns.set_style(style='white')
sns.set(rc={
    'figure.figsize': (12, 7),
    'axes.facecolor': 'white',
    'axes.grid': True, 'grid.color': '.9',
    'axes.linewidth': 1.0,
    'grid.linestyle': u'-'}, font_scale=1.5)
custom_colors = ["#3498db", "#95a5a6", "#34495e", "#2ecc71", "#e74c3c"]
sns.set_palette(custom_colors)

os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\FlightPreparence')

# Рабочие фреймы
df = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')
# Объединенный список из фреймов
comb = pd.concat([df, tsd], axis=0)

In [ ]:
#Преобразуем объектные столбцы в числовые 
df['date'] = df['date'].map({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
#Преобразуем объектные значения в колонке в числовые автоматически
df['date'] = LabelEncoder().fit_transform(df['date'])

In [ ]:
#Преобразуем Dtype колонки в числовые, если нужна внутренняя редактура
df['data'] = df['data'].str.replace(r'\D+', '') #Удаляем все НЕцифры
df['data'] = pd.to_numeric(df['data'], errors='coerce')

In [ ]:
#Извлекаем определенные фразы из объектных строк
for dataset in df:
    dataset['data_new'] = dataset['data'].str.extract(' ([A-Za-z]+)\.') #В данном случае слова, в конце которых стоит точка

In [ ]:
#Преобразуем float в int
df['data'] = pd.to_numeric(df['data'], downcast='integer')

In [ ]:
#Разбиваем колонки на новые, в каждой из которых только свои уникальные значения из прежней колонки
pd.get_dummies(df.Data, prefix="Emb", drop_first = True)

In [ ]:
# Выделяем числовые колонки в отдельную группу, получаем список заголовков колонок
num_feat = [x for x in comb.columns if comb[x].dtype !="object"]

In [ ]:
# Выделяем номинальные колонки в отдельную группу
cat_feat = [x for x in comb.columns if comb[x].dtype == "object"]

In [ ]:
#Выделяем упорядоченные колонки в отдельную группу (из номинальных, самостоятельно)
ord_feat = ['data','data1','data2']
cat_feat.remove(ord_feat)

In [ ]:
# Создаем список колонок, которые будут после обработки удалены из индекса. И добавляем в индекс новые
to_remove = []
to_remove.append('data')
num_feat.append('new_data')
ord_feat.append('new_data')
cat_feat.append('new_data')

In [ ]:
# Проверка на корреляцию всех числовых колонок
corr_matrix = comb[num_feat].corr()
plt.figure(figsize=(16,12))
sns.heatmap(corr_matrix.T, annot=True, cbar=False, cmap='coolwarm');

# Отдельно выделяем все, что выше 0.8
corr_matrix = comb[num_feat].corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr_matrix.T, annot=True, mask= corr_matrix < 0.8 ,cbar=False, cmap='coolwarm');

# Ищем колонки с наибольшей корреляцией предикторов к отклику, удаляем отклик
corr_ser = comb[num_feat].corr()['y'].sort_values(ascending=False).drop("y")
fig, ax = plt.subplots(figsize=(10,12))
sns.barplot(x=corr_ser.values, y=corr_ser.index, palette="rocket_r")
plt.title("Корреляция числовых предикторов с откликом");

# Удаление топ-корреляций из фреймов и из списка числовых колонок
high_correlated_var = ["data",'data1','data2']
comb = comb.drop(high_correlated_var, axis=1)

for c in high_correlated_var:
    num_feat.remove(c)

In [ ]:
#Замена пропущенных значений одной из переменных из колонки
df.Data.fillna(df.Data.mode()[0], inplace = True)

# Вариант 2, для категориальных колонок
for col in cat_feat:
    if comb[col].isna().sum() > 0:
        comb[col] = comb[col].fillna(value="NA") #Заменяем пробелы объектом 'NA'
    else:
        continue

In [ ]:
# Обработка категориальных колонок
new_data = comb['data'].apply(lambda x: 1 if x != "NA" else 0).astype("object") #Заменить все NA
new_data = comb['data'].apply(lambda x: x if x == "Predict" else "Other") #Разделить на доминирующий предикт и всех остальных

# Plot обновляемых данных
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,4))

sns.countplot(new_data, ax=axes[0])
sns.boxplot(x=new_data.values, y='y', data=comb, ax=axes[1])
axes[0].set_xlabel("New Data")
axes[0].set_ylabel("Y")
axes[1].set_xlabel("New Data")
axes[1].set_ylabel("Y");

In [ ]:
# Создание новой колонки на основе других (математические действия)
new_data = (comb['data1'].astype(int) - comb['data'].astype(int))
comb['new_data'] = pd.Series(new_data)

In [ ]:
# Обработка номинальных колонок

for col in ord_feat:  
    print(f" Column '{col}' has unique values {comb[col].unique()}")
    
ord_map = {"NA":0, "Po":1, "Fa":2, "TA":3, "Gd":4,"Ex":5}
for col in ord_feat:        
    if len(comb[col].unique()) <= 6 and col !="data":
        comb[col] = comb[col].map(ord_map)
        comb[col] = comb[col].astype(int)        
    elif col in ['data1', 'data2']:
        comb[col] = comb[col].astype(int)
    else:
        print('Dat all')

#Включаем обработанные упорядоченные в числовые
for col in ord_feat:
    num_feat.append(col)

In [ ]:
#Фильтр фрейма, оставить только строки с опредленным значением в одной из колонок
print( 'Before:', len(df) )
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]
df = df[df.Code.isin(gwa_codes)]
print( 'After:', len(df) )

In [ ]:
#Заполнить пустые значения средним по колонке
df['data'] = df.groupby(['data1']).Data.apply(lambda x: x.fillna(x.median()))

In [ ]:
# Cоздание новой колонки, аггрегирующей данные из нескольких
new_col = ['data','data1', 'data2', 'data3']
comb['new_data'] = np.zeros(len(comb)).reshape(len(comb),1) #Создаем новую пустую колонку длинной в список

#Переносим в новую колонку все значения из старых
for col in new_col:
    comb['new_data'] += comb[col] 
    
# Удаляем старые колонки из фрейма и из списка числовых колонок
to_remove = ['data','data1', 'data2', 'data3']
for c in to_remove:
    comb.drop(c, axis=1, inplace=True)
    num_feat.remove(c)

In [2]:
#Удаляем лишние колонки
df.drop(to_remove, inplace=True, axis=1)

#Или
for col in to_remove:
    comb.drop(col, axis=1, inplace=True)
    cat_feat.remove(col)

NameError: name 'to_remove' is not defined

In [ ]:
# Удалим первые 10 строк
df = df.drop(np.arange(10), axis=0)

In [ ]:
# Удалим всех пассажиров с переменной меньше 10 и больше 50 
df1 = df.drop(df[(df['data'] < 10) | (df['data'] > 50)].index)
df1.shape[0] / df.shape[0]

In [ ]:
#Категориальные колонки не имеют естественного порядка, поэтому преобразуем (еще методы: OneHotEncode и Label Encoding)
dummy_comb = pd.get_dummies(comb[cat_feat], drop_first=True)

for col in cat_feat:
    comb.drop(col, axis=1, inplace=True)
    
comb_with_dummies = pd.concat([comb, dummy_df], axis=1)

In [ ]:
#Преобразуем бинарные столбцы в численные. Колонку y тоже
df['data'] = df['data'].map({'predict1': 0, 'predict2': 1, 'predict3': 2})

In [ ]:
#Приводим множество названий колонок к типу set, находим разность двух множеств: 
print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

#Добавляем недостающую колонку. Смотрим, стоит ли склеивать отдельные переменные в более крупные классы
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)

#Проверяем совпадение колонок (если да, то True)
all(X_train.columns == X_test.columns)

In [ ]:
#Группировка колонки по двум другим переменным. Медиана группы подставлена в пропущенные строки 
grp = df.groupby(['data1', 'data2'])  
df.Data = grp.Data.apply(lambda x: x.fillna(x.median()))
df.Data.fillna(df.Data.median, inplace = True)

In [ ]:
#Абсорбируем и удаляем лишние знаки из объектов во всех строках колонки во фрейме и записываем их в новую колонку
for dataset in comb:
    dataset['data_new'] = df['data'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
#Построение облачного графика из объектов, где размер коррелирует с частотой
wc = WordCloud(width = 1000,height = 450,background_color = 'white').generate(str(df.Data_new.values))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

df.Data_new.value_counts()

In [ ]:
#Если в БД нет единой метрики, то стандартизируем данные
import math
from sklearn import preprocessing
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA

norm = preprocessing.StandardScaler()
norm.fit(df)
X = norm.transform(df)

#Cтандартизируем переменные 2
df_scaled = preprocessing.scale(df)

#Методом поиска главных компонентов проецируем данные на двумерную плоскость и получаем ранжирование компонентов по важности 
pca = PCA(n_components=5).fit(df_scaled) #Уточняем число компонент и источник данных 

#Доля разброса в данных, объясняемая главными компонентами
print('Влияние компонентов на общий разброс данных: ', pca.explained_variance_ratio_)

In [ ]:
# Возвращаем разделение данных на train/test
clean_train_df = comb_with_dummies[comb_with_dummies["Y"] > 0].copy()
clean_test_df = comb_with_dummies[comb_with_dummies["Y"].isna()].copy().drop("Y", axis=1, inplace=True)